# Models

This notebook covers everything related to Section 4 of the paper (Experiments and Results). This includes model training, hyperparameter search and generates the output files for the evaluation section. 

In [1]:
import torch
#import geopandas as gpd
import pandas as pd
import os
import datetime as dt
import time
from copy import deepcopy
import datetime
from shapely.geometry import Point, LineString, Polygon, asShape, mapping
import requests
import numpy as np
from shapely.ops import cascaded_union, transform
from functools import partial
import pyproj
#import folium
import math
import requests
import concurrent.futures
import json
#import plotly
from scipy import stats
import sklearn.metrics
import pickle
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib.colors import Normalize
from matplotlib import cm
#from matplotlib import cm, colors
#import seaborn as sn
#import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
os.chdir("../")
print(f"Current working directory: {os.getcwd()}")

Current working directory: /home/mwilbur/code/transit-energy-prediction/app


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
FEATURE_COLUMNS = ['precipitation_intensity', 'temperature', 'humidity', 'wind_speed', 'wind_gust', 'visibility', 'speed_meters_per_second', 'sr_ave_trip', 'jf_ave_trip', 'secondary', 'motorway_link', 'secondary_link', 'trunk', 'unclassified', 'motorway', 'residential', 'tertiary', 'primary_link', 'primary', 'elevation_diff', 'actual_elevation_change', 'distance_travelled_m']
TARGET = 'target_kg'

In [4]:
def get_point(row, pref):
    return Point(row[f"{pref}_stop_lon"], row[f"{pref}_stop_lat"])


def format_segments(df, add_stop_points=True, trip_id_format=False):
    df = df.set_geometry('geometry')
    df['trip_id'] = df['trip_id'].astype(int)
    df['segment_seq'] = df['segment_seq'].astype(int)
    df['start_stop_id'] = df['start_stop_id'].astype(int)
    df['end_stop_id'] = df['end_stop_id'].astype(int)
    df['direction_id'] = df['direction_id'].astype(int)
    df['route_id'] = df['route_id'].astype(str)
    df['distance_btw_stops'] = df['distance_btw_stops'].astype(float)
    df['start_stop_name'] = df['start_stop_name'].astype(str)
    df['end_stop_name'] = df['end_stop_name'].astype(str)
    df['distance_m'] = df['distance_m'].astype(float)
    df['gtfs_start_date'] = df['gtfs_start_date'].apply(lambda x: datetime.date.fromisoformat(x))
    df['gtfs_end_date'] = df['gtfs_end_date'].apply(lambda x: datetime.date.fromisoformat(x))
    df['segment_id'] = df['segment_id'].astype(str)
    
    df['XDSegID'] = df['XDSegID'].apply(lambda x: apply_format_list(x, col_type='int'))
    df['osm_ways'] = df['osm_ways'].apply(lambda x: apply_format_list(x, col_type='int'))
    df['tmc_id'] = df['tmc_id'].apply(lambda x: apply_format_list(x, col_type='string'))
    df['osm_way_fclasses'] = df['osm_way_fclasses'].apply(lambda x: apply_format_list(x, col_type='string'))
    df['elevation_list'] = df['elevation_list'].apply(lambda x: apply_format_list(x, col_type='float'))
    
    if add_stop_points:
        df['start_stop_geometry'] = df.apply(lambda row: get_point(row, 'start'), axis=1)
        df['end_stop_geometry'] = df.apply(lambda row: get_point(row, 'end'), axis=1)
        df = df.drop(columns=['start_stop_lon', 'start_stop_lat', 'end_stop_lat', 'end_stop_lon'])
    else:
        df['start_stop_lon'] = df['start_stop_lon'].astype(float)
        df['end_stop_lon'] = df['end_stop_lon'].astype(float)
        df['start_stop_lat'] = df['start_stop_lat'].astype(float)
        df['end_stop_lat'] = df['end_stop_lat'].astype(float)
    if trip_id_format:
        df['trip_id'] = df['trip_id'].apply(lambda x: int(str(int(x))[0:-3]))
    return df


def apply_format_list(x, col_type='int'):
    if isinstance(x, str):
        x = x.replace("[", "")
        x = x.replace("]", "")
        x = x.replace(" ", "")
        x = x.replace("'", "")
        if col_type == 'int':
            return [int(y) for y in x.split(",")]
        elif col_type == 'float':
            return [float(y) for y in x.split(",")]
        elif col_type == "string":
            return [str(y) for y in x.split(",")]
        else:
            return [y for y in x.split(",")]
            
    else:
        return x
    
    
def remove_trips_with_duplicates(df):
    print(f"Number of unique trips: {len(df['trip_id'].unique())}")
    trips_to_keep = []
    for trip_id in df['trip_id'].unique():
        temp = df[df['trip_id']==trip_id]
        if len(temp['start_stop_id'].unique()) == len(temp):
            trips_to_keep.append(trip_id)
    result = df[df['trip_id'].isin(trips_to_keep)]
    print(f"Number of trips without duplicate stops: {len(result['trip_id'].unique())}")
    return df[df['trip_id'].isin(trips_to_keep)]

In [ ]:
file_path = os.path.join(os.getcwd(), 'output_r', 'segments', 'segments.pkl') 
DF_SEG = pd.read_pickle(file_path)
DF_SEG = format_segments(DF_SEG, add_stop_points=True, trip_id_format=True)
DF_SEG = remove_trips_with_duplicates(DF_SEG)
DF_SEG['geometry_proj'] = DF_SEG.to_crs('EPSG:32616')['geometry']
DF_SEG.head(1)

In [5]:
file_path = os.path.join(os.getcwd(), 'output_r', 'training', 'data2.pkl')
DF = pd.read_pickle(file_path)
keep_cols = FEATURE_COLUMNS + ['target_kg_per_km', 'target_kg', 'vehicle_class']
DF = DF[keep_cols]
print(len(DF))
DF = DF.dropna()
print(len(DF))
DF['time_to_travel'] = DF.apply(lambda row: 1 / (row['speed_meters_per_second'] / row['distance_travelled_m']), axis=1)
DF = DF.drop(columns=['speed_meters_per_second'])
FEATURE_COLUMNS = ['precipitation_intensity', 'temperature', 'humidity', 'wind_speed', 'wind_gust', 'visibility', 'time_to_travel', 'sr_ave_trip', 'jf_ave_trip', 'secondary', 'motorway_link', 'secondary_link', 'trunk', 'unclassified', 'motorway', 'residential', 'tertiary', 'primary_link', 'primary', 'elevation_diff', 'actual_elevation_change', 'distance_travelled_m']
#DF['elevation_change'] = DF['actual_elevation_change']
#DF = DF.drop(columns=['actual_elevation_change'])
DF.head(1)

358861
350413


,precipitation_intensity,temperature,humidity,wind_speed,wind_gust,visibility,sr_ave_trip,jf_ave_trip,secondary,motorway_link,secondary_link,trunk,unclassified,motorway,residential,tertiary,primary_link,primary,elevation_diff,actual_elevation_change,distance_travelled_m,target_kg_per_km,target_kg,vehicle_class,time_to_travel
0,0.0,41.47,0.94,7.68,24.73,4.125,0.965237,0.24824,1,0,0,0,0,0,0,0,0,0,0.73,-0.54,234.224653,1.337093,0.31318,electric,32.0


## 1. Data Investigation

In [ ]:
DF['ave_speed'] = DF.apply(lambda row: (row['distance_travelled_m'] / row['time_to_travel']), axis=1)

In [ ]:
def get_energy_kwh(row):
    if row['vehicle_class'] == 'electric':
        return row['target_kg'] / 0.707
    else:
        return (row['target_kg'] / 10.18) * 37.95

DF['kwh'] = DF.apply(lambda row: get_energy_kwh(row), axis=1)

In [ ]:
print(f"Average diesel: {DF[DF['vehicle_class']=='diesel']['kwh'].mean()}, average hybrid {DF[DF['vehicle_class']=='hybrid']['kwh'].mean()}, average electric {DF[DF['vehicle_class']=='electric']['kwh'].mean()}")
print(f"median diesel: {DF[DF['vehicle_class']=='diesel']['kwh'].median()}, median hybrid {DF[DF['vehicle_class']=='hybrid']['kwh'].median()}, median electric {DF[DF['vehicle_class']=='electric']['kwh'].median()}")

In [ ]:
print(f"Average diesel: {DF[DF['vehicle_class']=='diesel']['target_kg_per_km'].mean()}, average hybrid {DF[DF['vehicle_class']=='hybrid']['target_kg_per_km'].mean()}, average electric {DF[DF['vehicle_class']=='electric']['target_kg_per_km'].mean()}")
print(f"median diesel: {DF[DF['vehicle_class']=='diesel']['target_kg_per_km'].median()}, median hybrid {DF[DF['vehicle_class']=='hybrid']['target_kg_per_km'].median()}, median electric {DF[DF['vehicle_class']=='electric']['target_kg_per_km'].median()}")

In [ ]:
print(f"Average diesel: {DF[DF['vehicle_class']=='diesel']['target_kg'].mean()}, average hybrid {DF[DF['vehicle_class']=='hybrid']['target_kg'].mean()}, average electric {DF[DF['vehicle_class']=='electric']['target_kg'].mean()}")
print(f"median diesel: {DF[DF['vehicle_class']=='diesel']['target_kg'].median()}, median hybrid {DF[DF['vehicle_class']=='hybrid']['target_kg'].median()}, median electric {DF[DF['vehicle_class']=='electric']['target_kg'].median()}")

In [ ]:
# generate boxplot values
result = {}
quantiles = [0.01, 0.25, 0.5, 0.75, 0.99]
for vehicle_type in DF['vehicle_class'].unique():
    result[f"ecr_{vehicle_type}"] = DF[DF['vehicle_class']==vehicle_type]['target_kg_per_km'].quantile(quantiles).tolist()
    result[f"ec_{vehicle_type}"] = DF[DF['vehicle_class']==vehicle_type]['target_kg'].quantile(quantiles).tolist()
    result[f"energy_{vehicle_type}"] = DF[DF['vehicle_class']==vehicle_type]['kwh'].quantile(quantiles).tolist()
df_results = pd.DataFrame(result)
out_path = os.path.join(os.getcwd(), "output_r", "latex", "energy_boxplot.csv")
df_results.to_csv(out_path, index=False)
df_results.head(1)

In [ ]:
#feature_cols = ['precipitation_intensity', 'temperature', 'humidity', 'wind_speed', 'wind_gust', 'visibility', 'speed_meters_per_second', 'sr_ave_segment', 'jf_ave_segment', 'sr_ave_trip', 'jf_ave_trip', 'elevation_diff', 'elevation_change', 'distance_travelled_m']
feature_cols = ['precipitation_intensity', 'temperature', 'humidity', 'wind_speed', 'wind_gust', 'visibility', 'time_to_travel', 'sr_ave_trip', 'jf_ave_trip', 'elevation_diff', 'actual_elevation_change', 'distance_travelled_m', 'ave_speed']
target = 'target_kg_per_km'

vehicle_types = DF['vehicle_class'].unique().tolist()
column_names = vehicle_types
results = []
index = []

for feature_col in feature_cols:
    result = []
    for vehicle_type in vehicle_types:
        c, p = stats.pearsonr(DF[DF['vehicle_class']==vehicle_type][feature_col], DF[DF['vehicle_class']==vehicle_type][target])
        result.append(c)
        #result.append(p)
    results.append(result)
    index.append(feature_col)
    
df_results = pd.DataFrame(results, index=index, columns=column_names)
#df_results.columns = ['ICEV', 'HV', 'EV']
df_results = df_results.sort_values(by=['diesel'])
df_results = df_results.round(3)
out_path = os.path.join(os.getcwd(), "output_r", "latex", "ecr_correlation2.csv")
df_results.to_csv(out_path, index=True)
df_results.head(1)

In [ ]:
#feature_cols = ['precipitation_intensity', 'temperature', 'humidity', 'wind_speed', 'wind_gust', 'visibility', 'speed_meters_per_second', 'sr_ave_segment', 'jf_ave_segment', 'sr_ave_trip', 'jf_ave_trip', 'elevation_diff', 'elevation_change', 'distance_travelled_m']
target = 'target_kg'

vehicle_types = DF['vehicle_class'].unique().tolist()
column_names = vehicle_types
results = []
index = []

for feature_col in feature_cols:
    result = []
    for vehicle_type in vehicle_types:
        c, p = stats.pearsonr(DF[DF['vehicle_class']==vehicle_type][feature_col], DF[DF['vehicle_class']==vehicle_type][target])
        result.append(c)
        #result.append(p)
    results.append(result)
    index.append(feature_col)
    
df_results = pd.DataFrame(results, index=index, columns=column_names)
#df_results.columns = ['ICEV', 'HV', 'EV']
df_results = df_results.sort_values(by=['diesel'])
df_results = df_results.round(7)
out_path = os.path.join(os.getcwd(), "output_r", "latex", "ec_correlation2.csv")
df_results.to_csv(out_path, index=True)
df_results.head(1)

In [ ]:
df_results

In [ ]:
DF = DF.drop(columns=['ave_speed'])

## Format Datasets

In [6]:
def train_val_test_split(df, val_set=False, random_state=102):
    if val_set is True:
        df_train = df.sample(frac=0.8, random_state=random_state)
        df_test = df.drop(df_train.index)
        df_val = df_train.sample(frac=0.1, random_state=random_state)
        df_train = df_train.drop(df_val.index)
        return df_train, df_val, df_test
    else:
        df_train = df.sample(frac=0.8, random_state=random_state)
        df_test = df.drop(df_train.index)
        return df_train, df_test
    

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        """
        features and labels should be pandas dataframes
        """
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        features = np.array(self.features.iloc[idx])
        labels = self.labels.iloc[idx]
        sample = {'features': features, 'labels': labels}
        return sample

In [7]:
df_diesel = DF[DF['vehicle_class']=='diesel']
df_hybrid = DF[DF['vehicle_class']=='hybrid']
df_electric = DF[DF['vehicle_class']=='electric']
total_samples = min([len(df_diesel), len(df_hybrid), len(df_electric)])
print([len(df_diesel), len(df_hybrid), len(df_electric)])
# get datasets
datasets = {}

temp = df_diesel.sample(n=total_samples, random_state=102).reset_index()
train, val, test = train_val_test_split(temp, val_set=True, random_state=102)
datasets['diesel_train'] = MyDataset(train[FEATURE_COLUMNS], train[TARGET])
datasets['diesel_val'] = MyDataset(val[FEATURE_COLUMNS], val[TARGET])
datasets['diesel_test'] = MyDataset(test[FEATURE_COLUMNS], test[TARGET])

temp = df_hybrid.sample(n=total_samples, random_state=102).reset_index()
train, val, test = train_val_test_split(temp, val_set=True, random_state=102)
datasets['hybrid_train'] = MyDataset(train[FEATURE_COLUMNS], train[TARGET])
datasets['hybrid_val'] = MyDataset(val[FEATURE_COLUMNS], val[TARGET])
datasets['hybrid_test'] = MyDataset(test[FEATURE_COLUMNS], test[TARGET])

temp = df_electric.sample(n=total_samples, random_state=102).reset_index()
train, val, test = train_val_test_split(temp, val_set=True, random_state=102)
datasets['electric_train'] = MyDataset(train[FEATURE_COLUMNS], train[TARGET])
datasets['electric_val'] = MyDataset(val[FEATURE_COLUMNS], val[TARGET])
datasets['electric_test'] = MyDataset(test[FEATURE_COLUMNS], test[TARGET])

for k, v in datasets.items():
    print(f"{k}: len features: {len(v.features)}, len target: {len(v.labels)}")

[110199, 198070, 42144]
diesel_train: len features: 30343, len target: 30343
diesel_val: len features: 3372, len target: 3372
diesel_test: len features: 8429, len target: 8429
hybrid_train: len features: 30343, len target: 30343
hybrid_val: len features: 3372, len target: 3372
hybrid_test: len features: 8429, len target: 8429
electric_train: len features: 30343, len target: 30343
electric_val: len features: 3372, len target: 3372
electric_test: len features: 8429, len target: 8429


## 2. MTL Model - parameter tuning

In [8]:
def save_checkpoint(state, is_best, filename='modelcheckpoint.pt'):
    """Save checkpoint if a new best is achieved"""
    if is_best:
        #print ("=> Saving a new best")
        torch.save(state, filename)  # save checkpoint
    else:
        #print ("=> Validation Accuracy did not improve")
        pass

In [9]:
class MultiTaskModel(torch.nn.Module):
    def __init__(self, 
                 n_feature,
                 n_shared_width,
                 n_shared_depth, 
                 n_task_widths, 
                 outlayer='linear'):
        super(MultiTaskModel, self).__init__()
        self.n_feature = n_feature
        self.n_shared_width = n_shared_width
        self.n_shared_depth = n_shared_depth
        self.n_task_widths = n_task_widths
        self.outlayer = outlayer
        #print(self.outlayer)
        
        self.task = None # should be set to 'hybrid', 'electric', or 'diesel'
        
        self.s1 = torch.nn.Linear(self.n_feature, self.n_shared_width)
        self.s2 = torch.nn.Linear(self.n_shared_width, self.n_shared_width)
        self.s3 = torch.nn.Linear(self.n_shared_width, self.n_shared_width)
        self.s4 = torch.nn.Linear(self.n_shared_width, self.n_shared_width)
        self.s5 = torch.nn.Linear(self.n_shared_width, self.n_shared_width)
        self.s6 = torch.nn.Linear(self.n_shared_width, self.n_shared_width)
        
        self.d1 = torch.nn.Linear(self.n_shared_width, self.n_task_widths[0])
        self.d2 = torch.nn.Linear(self.n_task_widths[0], self.n_task_widths[1])
        self.d3 = torch.nn.Linear(self.n_task_widths[1], self.n_task_widths[2])
        self.dout = torch.nn.Linear(self.n_task_widths[2], 1)
        
        self.e1 = torch.nn.Linear(self.n_shared_width, self.n_task_widths[0])
        self.e2 = torch.nn.Linear(self.n_task_widths[0], self.n_task_widths[1])
        self.e3 = torch.nn.Linear(self.n_task_widths[1], self.n_task_widths[2])
        self.eout = torch.nn.Linear(self.n_task_widths[2], 1)
        
        self.h1 = torch.nn.Linear(self.n_shared_width, self.n_task_widths[0])
        self.h2 = torch.nn.Linear(self.n_task_widths[0], self.n_task_widths[1])
        self.h3 = torch.nn.Linear(self.n_task_widths[1], self.n_task_widths[2])
        self.hout = torch.nn.Linear(self.n_task_widths[2], 1)
    
    def forward(self, x):
        # shared layers
        x = torch.nn.functional.relu(self.s1(x))
        if self.n_shared_depth > 1:
            x = torch.nn.functional.relu(self.s2(x))
        if self.n_shared_depth > 2:
            x = torch.nn.functional.relu(self.s3(x))
        if self.n_shared_depth > 3:
            x = torch.nn.functional.relu(self.s4(x))
        if self.n_shared_depth > 4:
            x = torch.nn.functional.relu(self.s5(x))
        if self.n_shared_depth > 5:
            x = torch.nn.functional.relu(self.s6(x))
        
        # task specific layers
        if self.task == 'diesel':
            x = torch.nn.functional.relu(self.d1(x))
            x = torch.nn.functional.relu(self.d2(x))
            x = torch.nn.functional.relu(self.d3(x))
            x = self.dout(x)
            if self.outlayer == 'relu':
                x = torch.nn.functional.relu(x)
        if self.task == 'hybrid':
            x = torch.nn.functional.relu(self.h1(x))
            x = torch.nn.functional.relu(self.h2(x))
            x = torch.nn.functional.relu(self.h3(x))
            x = self.hout(x)
            if self.outlayer == 'relu':
                x = torch.nn.functional.relu(x)
        if self.task == 'electric':
            x = torch.nn.functional.relu(self.e1(x))
            x = torch.nn.functional.relu(self.e2(x))
            x = torch.nn.functional.relu(self.e3(x))
            x = self.eout(x)
        return torch.flatten(x)

    
def mean_absolute_percentage_error(model, dataset):
    features = torch.tensor(dataset.features.values).double().to(device)
    y_pred = model(features).cpu().detach().numpy()
    y_true = dataset.labels.values
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


def get_loss_multitask(model, dataset, vehicle_type, loss='mse'):
    model.task = vehicle_type
    if loss == 'mse':
        loss_function = torch.nn.MSELoss(reduction='mean')
    else:
        loss_function = torch.nn.L1Loss(reduction='mean')
    features = torch.tensor(dataset.features.values).double().to(device)
    labels = torch.tensor(dataset.labels.values).double().to(device)
    #features = torch.tensor(dataset.features.values).double()
    #labels = torch.tensor(dataset.labels.values).double()
    predicts = model(features)
    return loss_function(predicts, labels).item()

import GPUtil

def get_loss_multitask_dataloader(model, dataloader, vehicle_type, loss='mse'):
    result = 0
    model.task = vehicle_type
    import GPUtil
    if loss == 'mse':
        loss_function = torch.nn.MSELoss(reduction='sum')
    elif loss == 'huber':
        loss_function = torch.nn.SmoothL1Loss(reduction='sum')
    else:
        loss_function = torch.nn.L1Loss(reduction='sum')
    for idx, batch_data in enumerate(dataloader):
        features = batch_data['features'].double().to(device)
        labels = batch_data['labels'].double().to(device)
        predicts = model(features)
        result += float(loss_function(predicts, labels).item())
    return result / len(dataloader.dataset)


def gpu_memory():
    tt = torch.cuda.get_device_properties(0).total_memory
    rr = torch.cuda.memory_reserved(0) 
    aa = torch.cuda.memory_allocated(0)
    ff = rr-aa  # free inside reserved
    print(ff / 1000000)


def train_multitask(datasets, 
                    n_shared_width,
                    n_shared_depth,
                    n_task_widths,
                    lr, 
                    batch_size, 
                    num_epochs, 
                    device, 
                    loss_fun='mse', 
                    test=False, 
                    stats_at_epoch=10, 
                    early_stopping=True, 
                    outlayer='linear', 
                    early_stop_epochs=1, 
                    metrics=['mse', 'mae', 'huber'], 
                    opt='adam', 
                    filename='modelcheckpoint.pt', 
                    val_on_train=True):
    
    # setup dataloaders
    diesel_train_dataloader = torch.utils.data.DataLoader(datasets['diesel_train'], batch_size=batch_size, shuffle=True, num_workers=8)
    hybrid_train_dataloader = torch.utils.data.DataLoader(datasets['hybrid_train'], batch_size=batch_size, shuffle=True, num_workers=8)
    electric_train_dataloader = torch.utils.data.DataLoader(datasets['electric_train'], batch_size=batch_size, shuffle=True, num_workers=8)
    if test is False:
        diesel_test_dataloader = torch.utils.data.DataLoader(datasets['diesel_val'], batch_size=batch_size, shuffle=True, num_workers=8)
        hybrid_test_dataloader = torch.utils.data.DataLoader(datasets['hybrid_val'], batch_size=batch_size, shuffle=True, num_workers=8)
        electric_test_dataloader = torch.utils.data.DataLoader(datasets['electric_val'], batch_size=batch_size, shuffle=True, num_workers=8)
    else:
        diesel_test_dataloader = torch.utils.data.DataLoader(datasets['diesel_test'], batch_size=batch_size, shuffle=True, num_workers=8)
        hybrid_test_dataloader = torch.utils.data.DataLoader(datasets['hybrid_test'], batch_size=batch_size, shuffle=True, num_workers=8)
        electric_test_dataloader = torch.utils.data.DataLoader(datasets['electric_test'], batch_size=batch_size, shuffle=True, num_workers=8)
    
    # setup model
    model = MultiTaskModel(len(FEATURE_COLUMNS), n_shared_width, n_shared_depth, n_task_widths, outlayer=outlayer).to(device)
    if loss_fun == 'mse':
        loss_function = torch.nn.MSELoss(reduction='mean')
    elif loss_fun == 'huber':
        loss_function = torch.nn.SmoothL1Loss(reduction='mean')
    else:
        loss_function = torch.nn.L1Loss(reduction='mean')
    
    if opt == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    else:
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    model.double()
    
    # store results
    losses = {}
    for metric in metrics:
        losses[f"diesel_test_{metric}"] = []
        losses[f"hybrid_test_{metric}"] = []
        losses[f"electric_test_{metric}"] = []
    losses['diesel_train_mse'] = []
    losses['hybrid_train_mse'] = []
    losses['electric_train_mse'] = []
    
    model.eval()
    with torch.no_grad():
        for metric in metrics:
            losses[f"diesel_test_{metric}"].append(get_loss_multitask_dataloader(model, diesel_test_dataloader, 'diesel', loss=metric))
            losses[f"hybrid_test_{metric}"].append(get_loss_multitask_dataloader(model, hybrid_test_dataloader, 'hybrid', loss=metric))
            losses[f"electric_test_{metric}"].append(get_loss_multitask_dataloader(model, electric_test_dataloader, 'electric', loss=metric))
            
        losses["diesel_train_mse"].append(get_loss_multitask_dataloader(model, diesel_train_dataloader, 'diesel', loss='mse'))
        losses["hybrid_train_mse"].append(get_loss_multitask_dataloader(model, hybrid_train_dataloader, 'hybrid', loss='mse'))
        losses["electric_train_mse"].append(get_loss_multitask_dataloader(model, electric_train_dataloader, 'electric', loss='mse'))

    counter = 0
    best_loss = losses[f"diesel_test_{loss_fun}"][-1] + losses[f"hybrid_test_{loss_fun}"][-1] + losses[f"electric_test_{loss_fun}"][-1]
    for epoch in range(num_epochs):
        for i, batch_data in enumerate(zip(diesel_train_dataloader, hybrid_train_dataloader, electric_train_dataloader)):
            model.train()
            
            model.task = 'diesel'
            features = batch_data[0]['features'].double().to(device)
            labels = batch_data[0]['labels'].double().to(device)
            #optimizer.zero_grad()
            predicts = model(features)
            loss = loss_function(predicts, labels)
            loss.backward()
            #optimizer.step()
            
            model.task = 'hybrid'
            features = batch_data[1]['features'].double().to(device)
            labels = batch_data[1]['labels'].double().to(device)
            #optimizer.zero_grad()
            predicts = model(features)
            loss = loss_function(predicts, labels)
            loss.backward()
            #optimizer.step()
            
            model.task = 'electric'
            features = batch_data[2]['features'].double().to(device)
            labels = batch_data[2]['labels'].double().to(device)
            #optimizer.zero_grad()
            predicts = model(features)
            loss = loss_function(predicts, labels)
            loss.backward()
            
            optimizer.step()
            optimizer.zero_grad()
        
        if (epoch % stats_at_epoch == 0):
            model.eval()
            with torch.no_grad():
                for metric in metrics:
                    losses[f"diesel_test_{metric}"].append(get_loss_multitask_dataloader(model, diesel_test_dataloader, 'diesel', loss=metric))
                    losses[f"hybrid_test_{metric}"].append(get_loss_multitask_dataloader(model, hybrid_test_dataloader, 'hybrid', loss=metric))
                    losses[f"electric_test_{metric}"].append(get_loss_multitask_dataloader(model, electric_test_dataloader, 'electric', loss=metric))

                losses["diesel_train_mse"].append(get_loss_multitask_dataloader(model, diesel_train_dataloader, 'diesel', loss='mse'))
                losses["hybrid_train_mse"].append(get_loss_multitask_dataloader(model, hybrid_train_dataloader, 'hybrid', loss='mse'))
                losses["electric_train_mse"].append(get_loss_multitask_dataloader(model, electric_train_dataloader, 'electric', loss='mse'))
            if val_on_train is False:
                new_loss = losses[f"diesel_test_{loss_fun}"][-1] + losses[f"hybrid_test_{loss_fun}"][-1] + losses[f"electric_test_{loss_fun}"][-1]
            else:
                new_loss = losses[f"diesel_train_mse"][-1] + losses[f"hybrid_train_mse"][-1] + losses[f"electric_train_mse"][-1]
            #print(new_loss)
            if new_loss > best_loss:
                counter += 1
            else:
                counter = 0
                state = {'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}
                save_checkpoint(state, True, filename=filename)
                best_loss = deepcopy(new_loss)
            if (counter >= early_stop_epochs) and (epoch >= 100) and (early_stopping is True):
                break
        
    df_results = pd.DataFrame(losses)
    for metric in metrics:
        df_results[f"total_test_{metric}"] = df_results.apply(lambda row: row[f"diesel_test_{metric}"] + row[f"hybrid_test_{metric}"] + row[f"electric_test_{metric}"], axis=1)
    df_results[f"total_train_mse"] = df_results.apply(lambda row: row["diesel_train_mse"] + row["hybrid_train_mse"] + row["electric_train_mse"], axis=1)
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['model_state_dict'])
    return df_results, model


In [13]:
# parameter tuning - MTL model
        
outlayer = 'linear'
opt = 'adamw'
n_shared_width_list = [200, 300, 400]
n_shared_depth_list = [3, 4, 5]
n_task_widths = [64, 32, 16]
lr_list = [0.01, 0.005, 0.001, 0.0005, 0.0001]
#lr_list = [0.001, 0.0005, 0.0001]
batch_size_list = [128, 256, 512]
loss_fun_list = ['mse']
metrics = ['mse']
dropout = 0

num_epochs = 150
early_stopping = True
stats_at_epoch = 1
test = False
early_stop_epochs = 10
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda:1')

results = {'test_mse': [],  
           'n_shared_width': [], 
           'n_shared_depth': [], 
           'lr': [], 
           'batch_size': [], 
           'loss_fun': []}

out_path = os.path.join(os.getcwd(), 'output_r', 'latex', 'mtl_gridsearch.csv')

for n_shared_width in n_shared_width_list:
    for n_shared_depth in n_shared_depth_list:
            for lr in lr_list:
                for batch_size in batch_size_list:
                    for loss_fun in loss_fun_list:
                        if os.path.isfile(out_path):
                            df_past = pd.read_csv(out_path)
                            if len(df_past[(df_past['n_shared_width']==n_shared_width) & (df_past['n_shared_depth']==n_shared_depth) & (df_past['lr']==lr) & (df_past['batch_size']==batch_size) & (df_past['loss_fun']==loss_fun)]) != 0:
                                print('continue')
                                continue
                        else:
                            df_past = None
                                
                        df_results, _ = train_multitask(datasets, 
                                                        n_shared_width, 
                                                        n_shared_depth, 
                                                        n_task_widths, 
                                                        lr, 
                                                        batch_size, 
                                                        num_epochs, 
                                                        device, 
                                                        loss_fun=loss_fun, 
                                                        test=test, 
                                                        stats_at_epoch=stats_at_epoch, 
                                                        early_stopping=early_stopping, 
                                                        outlayer=outlayer, 
                                                        early_stop_epochs=early_stop_epochs, 
                                                        metrics=metrics, 
                                                        opt=opt)
                            
                        results['test_mse'].append(df_results['total_test_mse'].min())
                        results['n_shared_width'].append(n_shared_width)
                        results['n_shared_depth'].append(n_shared_depth)
                        results['lr'].append(lr)
                        results['batch_size'].append(batch_size)
                        results['loss_fun'].append(loss_fun)
                            
                        df_new = pd.DataFrame(results)
                        if df_past is not None:
                            df_new = pd.concat([df_past, df_new], ignore_index=True)
                            df_new = df_new.drop_duplicates(subset=['n_shared_width', 'n_shared_depth', 'lr', 'batch_size', 'loss_fun'])
                        df_new.to_csv(out_path, index=False)
                        print('finished a model')
        

finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a model
finished a mod

## 3. Baseline Model - parameter tuning

In [11]:
class BaselineModel(torch.nn.Module):
    def __init__(self, 
                 n_feature,
                 n_hidden,
                 n_output,
                 dropout,
                 n_layers):
        super(BaselineModel, self).__init__()
        self.n_feature = n_feature
        self.n_hidden = n_hidden
        self.n_output = n_output
        self.dropout = dropout
        self.n_layers = n_layers
        
        self.f1 = torch.nn.Linear(self.n_feature, self.n_hidden)
        self.f2 = torch.nn.Linear(self.n_hidden, self.n_hidden)
        self.f3 = torch.nn.Linear(self.n_hidden, self.n_hidden)
        self.f4 = torch.nn.Linear(self.n_hidden, self.n_hidden)
        self.f5 = torch.nn.Linear(self.n_hidden, self.n_hidden)
        self.f6 = torch.nn.Linear(self.n_hidden, self.n_hidden)
        #self.out = torch.nn.Linear(self.n_hidden, self.n_output)
        
        self.v1 = torch.nn.Linear(self.n_hidden, 64)
        self.v2 = torch.nn.Linear(64, 32)
        self.v3 = torch.nn.Linear(32, 16)
        self.out = torch.nn.Linear(16, self.n_output)
    
    def feature_extractor(self, x):
        x = torch.nn.functional.relu(self.f1(x))
        x = torch.nn.functional.dropout(x, p=self.dropout, training=self.training)
        if self.n_layers > 1:
            x = torch.nn.functional.relu(self.f2(x))
            x = torch.nn.functional.dropout(x, p=self.dropout, training=self.training)
        if self.n_layers > 2:
            x = torch.nn.functional.relu(self.f3(x))
            x = torch.nn.functional.dropout(x, p=self.dropout, training=self.training)
        if self.n_layers > 3:
            x = torch.nn.functional.relu(self.f4(x))
            x = torch.nn.functional.dropout(x, p=self.dropout, training=self.training)
        if self.n_layers > 4:
            x = torch.nn.functional.relu(self.f5(x))
            x = torch.nn.functional.dropout(x, p=self.dropout, training=self.training)
        if self.n_layers > 5:
            x = torch.nn.functional.relu(self.f6(x))
            x = torch.nn.functional.dropout(x, p=self.dropout, training=self.training)
        return x
    
    def regressor(self, x):
        x = torch.nn.functional.relu(self.v1(x))
        x = torch.nn.functional.relu(self.v2(x))
        x = torch.nn.functional.relu(self.v3(x))
        x = self.out(x)   
        return x
    
    
    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.regressor(x)
        return torch.flatten(x)
    
    def forwardVoid(self, x):
        x = torch.nn.functional.relu(self.f1(x))
        x = torch.nn.functional.dropout(x, p=self.dropout, training=self.training)
        if self.n_layers > 1:
            x = torch.nn.functional.relu(self.f2(x))
            x = torch.nn.functional.dropout(x, p=self.dropout, training=self.training)
        if self.n_layers > 2:
            x = torch.nn.functional.relu(self.f3(x))
            x = torch.nn.functional.dropout(x, p=self.dropout, training=self.training)
        if self.n_layers > 3:
            x = torch.nn.functional.relu(self.f4(x))
            x = torch.nn.functional.dropout(x, p=self.dropout, training=self.training)
        if self.n_layers > 4:
            x = torch.nn.functional.relu(self.f5(x))
            x = torch.nn.functional.dropout(x, p=self.dropout, training=self.training)
        if self.n_layers > 5:
            x = torch.nn.functional.relu(self.f6(x))
            x = torch.nn.functional.dropout(x, p=self.dropout, training=self.training)
        
        x = torch.nn.functional.relu(self.v1(x))
        x = torch.nn.functional.relu(self.v2(x))
        x = torch.nn.functional.relu(self.v3(x))
        x = self.out(x)         
        return torch.flatten(x)
    

def get_loss_baseline_dataloader(model, dataloader, loss='mse'):
    result = 0
    if loss == 'mse':
        loss_function = torch.nn.MSELoss(reduction='sum')
    elif loss == 'huber':
        loss_function = torch.nn.SmoothL1Loss(reduction='sum')
    else:
        loss_function = torch.nn.L1Loss(reduction='sum')
    for idx, batch_data in enumerate(dataloader):
        features = batch_data['features'].double().to(device)
        labels = batch_data['labels'].double().to(device)
        predicts = model(features)
        result += float(loss_function(predicts, labels).item())
    return result / len(dataloader.dataset)
    
    
def train_baseline(datasets,
                   vehicle_class,
                   n_hidden, 
                   n_layers, 
                   lr, 
                   batch_size, 
                   num_epochs, 
                   device, 
                   dropout=0,
                   test=False, 
                   stats_at_epoch=10, 
                   early_stopping=True, 
                   early_stop_epochs=1, 
                   metrics=['mse', 'mae', 'huber'], 
                   loss_fun='mse', 
                   filename='modelcheckpoint.pt', 
                   val_on_train=True, 
                   opt='adam', 
                   model=None, 
                   optimizer=None):
    # setup dataloaders
    train_dataloader = torch.utils.data.DataLoader(datasets[f"{vehicle_class}_train"], batch_size=batch_size, shuffle=True, num_workers=8)
    if test is True:
        test_dataloader = torch.utils.data.DataLoader(datasets[f"{vehicle_class}_test"], batch_size=batch_size, shuffle=True, num_workers=8)
    else:
        test_dataloader = torch.utils.data.DataLoader(datasets[f"{vehicle_class}_val"], batch_size=batch_size, shuffle=True, num_workers=8)
    
    # setup model
    if model is None:
        model = BaselineModel(len(FEATURE_COLUMNS), n_hidden, 1, dropout, n_layers).to(device)
    loss_function = torch.nn.MSELoss()
    
    if optimizer is None:
        if opt == 'adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        else:
            optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    model.double()
    
    # store results
    losses = {}
    for metric in metrics:
        losses[f"test_{metric}"] = []
    losses["train_mse"] = []
    
    # get initial loss
    counter = 0
    model.eval()
    with torch.no_grad():
        for metric in metrics:
            losses[f"test_{metric}"].append(get_loss_baseline_dataloader(model, test_dataloader, loss=metric))
    losses["train_mse"].append(get_loss_baseline_dataloader(model, train_dataloader, loss='mse'))
    best_loss = losses[f"test_{loss_fun}"][-1]
    
    # train
    for epoch in range(num_epochs):
        model.train()
        for idx, batch_data in enumerate(train_dataloader):
            features = batch_data['features'].double().to(device)
            labels = batch_data['labels'].double().to(device)
            optimizer.zero_grad()
            predicts = model(features)
            loss = loss_function(predicts, labels)
            loss.backward()
            optimizer.step()
        
        if (epoch % stats_at_epoch == 0):
            model.eval()
            with torch.no_grad():
                for metric in metrics:
                    losses[f"test_{metric}"].append(get_loss_baseline_dataloader(model, test_dataloader, loss=metric))
                losses["train_mse"].append(get_loss_baseline_dataloader(model, train_dataloader, loss='mse'))

            if val_on_train is False:
                new_loss = losses[f"test_{loss_fun}"][-1] 
            else:
                new_loss = losses["train_mse"][-1] 
                
            if new_loss > best_loss:
                counter += 1
            else:
                counter = 0
                state = {'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}
                save_checkpoint(state, True, filename=filename)
                best_loss = deepcopy(new_loss)
            if (counter >= early_stop_epochs) and (epoch >= 75) and (early_stopping is True):
                break
    df_results = pd.DataFrame(losses)
    
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['model_state_dict'])
    return df_results, model


def train_baseline_allclasses(datasets,
                              n_hidden, 
                              n_layers, 
                              lr, 
                              batch_size, 
                              num_epochs, 
                              device, 
                              dropout=0,
                              test=False,
                              vehicle_class_list=['diesel', 'hybrid', 'electric'],
                              stats_at_epoch=10, 
                              early_stopping=True, 
                              early_stop_epochs=1, 
                              metrics=['mse', 'mae', 'huber'], 
                              loss_fun='mse', 
                              filename='baseline.pt', 
                              val_on_train=True, 
                              opt='adam'):
    # store results
    losses = {}
    for metric in metrics:
        for vehicle_class in vehicle_class_list:
            losses[f"{vehicle_class}_test_{metric}"] = []
    for vehicle_class in vehicle_class_list:
        losses[f"{vehicle_class}_train_mse"] = []
    
    # train and store results
    models = {}
    for vehicle_class in vehicle_class_list:
        df_result, model = train_baseline(datasets, 
                                          vehicle_class, 
                                          n_hidden, 
                                          n_layers, 
                                          lr, 
                                          batch_size, 
                                          num_epochs, 
                                          device, 
                                          dropout=0, 
                                          test=test, 
                                          stats_at_epoch=stats_at_epoch, 
                                          early_stopping=early_stopping, 
                                          early_stop_epochs=early_stop_epochs, 
                                          metrics=metrics, 
                                          loss_fun=loss_fun, 
                                          filename=filename, 
                                          val_on_train=val_on_train, 
                                          opt=opt)
        models[vehicle_class] = deepcopy(model)
        for metric in metrics:
            losses[f"{vehicle_class}_test_{metric}"] = df_result[f"test_{metric}"]
        losses[f"{vehicle_class}_train_mse"] = df_result["train_mse"]
    
    # add total values to result
    df_results = pd.DataFrame(losses)
    for metric in metrics:
         df_results[f"total_test_{metric}"] = df_results.apply(lambda row: row[f"diesel_test_{metric}"] + row[f"hybrid_test_{metric}"] + row[f"electric_test_{metric}"], axis=1)
    df_results[f"total_train_mse"] = df_results.apply(lambda row: row[f"diesel_train_mse"] + row[f"hybrid_train_mse"] + row[f"electric_train_mse"], axis=1)
    return df_results, models
            

In [ ]:
# Baseline hyper parameter turning

n_hidden_list = [300, 400, 500]
n_layers_list = [3, 4, 5]
#lr_list = [0.01, 0.005, 0.001, 0.0005, 0.0001]
lr_list = [0.001, 0.0005, 0.0001]
batch_size_list = [128, 256, 512]

num_epochs = 50
early_stopping = True
stats_at_epoch = 1
test = False
early_stop_epochs = 10
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda:1')
metrics = ['mse']
loss_function = 'mse'

results = {'test_mse': [],
           'diesel_test_mse': [],
           'hybrid_test_mse': [],
           'electric_test_mse': [],
           'n_hidden': [], 
           'n_layers': [], 
           'lr': [], 
           'batch_size': []}

out_path = os.path.join(os.getcwd(), 'output_r', 'latex', 'baseline_gridsearch.csv')

for n_hidden in n_hidden_list:
    for n_layers in n_layers_list:
        for lr in lr_list:
            for batch_size in batch_size_list:
                if os.path.isfile(out_path):
                    df_past = pd.read_csv(out_path)
                    if len(df_past[(df_past['n_hidden']==n_hidden) & (df_past['n_layers']==n_layers) & (df_past['lr']==lr) & (df_past['batch_size']==batch_size)]) != 0:
                        print('continue')
                        continue
                else:
                    df_past = None
                
                df_results, models = train_baseline_allclasses(datasets, 
                                                       n_hidden, 
                                                       n_layers, 
                                                       lr, 
                                                       batch_size, 
                                                       num_epochs, 
                                                       device, 
                                                       dropout=0, 
                                                       test=test, 
                                                       stats_at_epoch=stats_at_epoch, 
                                                       early_stopping=early_stopping, 
                                                       early_stop_epochs=early_stop_epochs, 
                                                       metrics=metrics, 
                                                       loss_fun=loss_function)
                #results['test_huber'].append(df_results['total_test_huber'].min())
                results['test_mse'].append(df_results['total_test_mse'].min())
                results['diesel_test_mse'].append(df_results['diesel_test_mse'].min())
                results['hybrid_test_mse'].append(df_results['hybrid_test_mse'].min())
                results['electric_test_mse'].append(df_results['electric_test_mse'].min())
                results['n_hidden'].append(n_hidden)
                results['n_layers'].append(n_layers)
                results['lr'].append(lr)
                results['batch_size'].append(batch_size)
            
                df_new = pd.DataFrame(results)
                if df_past is not None:
                    df_new = pd.concat([df_past, df_new], ignore_index=True)
                    df_new = df_new.drop_duplicates()
                df_new.to_csv(out_path, index=False)
                print("Done with a model")
                

Done with a model
Done with a model
Done with a model
Done with a model
Done with a model
Done with a model
Done with a model
Done with a model
Done with a model
Done with a model
Done with a model
Done with a model
Done with a model
Done with a model
Done with a model


## 4. MTL vs Baseline

This section outputs the results for Section 4.2 of the paper.

In [ ]:
mtl_params = {'loss_fun': 'mse', 'outlayer': 'linear', 'n_shared_width': 300, 'n_shared_depth': 5, 'n_task_widths': [64, 32, 16], 'lr': 0.0005, 'batch_size': 256, 'opt': 'adamw', 'filename': os.path.join(os.getcwd(), 'models', 'mtlvsbaselinemtl.pt')}
baseline_params = {'loss_fun': 'mse', 'n_hidden': 300, 'n_layers': 5, 'lr': 0.0005, 'batch_size': 256, 'filename': os.path.join(os.getcwd(), 'models', 'mtlvsbaselinebaseline.pt')}

num_epochs = 150
device = torch.device('cuda:1')
test = True
stats_at_epoch = 1
early_stopping = False
early_stop_epochs = 10
dropout = 0
number_of_runs = 10

In [ ]:
# train multitask
results = {}

losses = {'diesel_test_mse': [], 
          'hybrid_test_mse': [], 
          'electric_test_mse': [], 
          'diesel_test_mae': [], 
          'hybrid_test_mae': [], 
          'electric_test_mae': [],
          'diesel_test_huber': [], 
          'hybrid_test_huber': [], 
          'electric_test_huber': []}

for run in range(number_of_runs):
    start_time = time.time()
    df_results, model = train_multitask(datasets, 
                                        mtl_params['n_shared_width'], 
                                        mtl_params['n_shared_depth'], 
                                        mtl_params['n_task_widths'],
                                        mtl_params['lr'], 
                                        mtl_params['batch_size'], 
                                        num_epochs, 
                                        device, 
                                        loss_fun=mtl_params['loss_fun'], 
                                        test=test, 
                                        stats_at_epoch=stats_at_epoch, 
                                        early_stopping=early_stopping, 
                                        outlayer=mtl_params['outlayer'], 
                                        early_stop_epochs=early_stop_epochs, 
                                        metrics=['mse', 'mae', 'huber'], 
                                        opt=mtl_params['opt'], 
                                        filename=mtl_params['filename'], 
                                        val_on_train=True)
    
    ind = df_results[['total_test_huber']].idxmin()['total_test_huber']
    for k in losses.keys():
        losses[k].append(df_results.iloc[ind][k])
    print(time.time() - start_time)
    
out_path = os.path.join(os.getcwd(), 'output_r', 'latex', 'mtltrainingloss.csv')
df_results.to_csv(out_path, index_label='epoch')

temp = pd.DataFrame(losses)
temp['total_test_huber'] = temp.apply(lambda row: row['diesel_test_huber'] + row['hybrid_test_huber'] + row['electric_test_huber'], axis=1)
temp['total_test_mse'] = temp.apply(lambda row: row['diesel_test_mse'] + row['hybrid_test_mse'] + row['electric_test_mse'], axis=1)
temp['total_test_mae'] = temp.apply(lambda row: row['diesel_test_mae'] + row['hybrid_test_mae'] + row['electric_test_mae'], axis=1)
results['mtl_mean'] = temp.mean(axis=0)
results['mtl_median'] = temp.median(axis=0)

In [ ]:
losses = {'diesel_test_mse': [], 
          'hybrid_test_mse': [], 
          'electric_test_mse': [], 
          'diesel_test_mae': [], 
          'hybrid_test_mae': [], 
          'electric_test_mae': [],
          'diesel_test_huber': [], 
          'hybrid_test_huber': [], 
          'electric_test_huber': []}

for run in range(number_of_runs):
    start_time = time.time()
    df_results, models = train_baseline_allclasses(datasets, 
                                                   baseline_params['n_hidden'], 
                                                   baseline_params['n_layers'], 
                                                   baseline_params['lr'], 
                                                   baseline_params['batch_size'], 
                                                   num_epochs, 
                                                   device, 
                                                   dropout=0, 
                                                   test=test, 
                                                   stats_at_epoch=stats_at_epoch, 
                                                   early_stopping=early_stopping, 
                                                   early_stop_epochs=early_stop_epochs, 
                                                   metrics=['mse', 'mae', 'huber'], 
                                                   loss_fun='mse', 
                                                   filename=baseline_params['filename'])
    
    ind = df_results[['total_test_mse']].idxmin()['total_test_mse']
    for k in losses.keys():
        losses[k].append(df_results.iloc[ind][k])
        
    #for k in losses.keys():
    #    losses[k].append(df_results[k].min())
    print(time.time() - start_time)
    
out_path = os.path.join(os.getcwd(), 'output_r', 'latex', 'baselinetrainingloss.csv')
df_results.to_csv(out_path, index_label='epoch')

temp = pd.DataFrame(losses)
temp['total_test_huber'] = temp.apply(lambda row: row['diesel_test_huber'] + row['hybrid_test_huber'] + row['electric_test_huber'], axis=1)
temp['total_test_mse'] = temp.apply(lambda row: row['diesel_test_mse'] + row['hybrid_test_mse'] + row['electric_test_mse'], axis=1)
temp['total_test_mae'] = temp.apply(lambda row: row['diesel_test_mae'] + row['hybrid_test_mae'] + row['electric_test_mae'], axis=1)
results['baseline_mean'] = temp.mean(axis=0)
results['baseline_median'] = temp.median(axis=0)

In [ ]:
temp = pd.DataFrame(results)
out_path = os.path.join(os.getcwd(), 'output_r', 'latex', 'mtl_vs_baseline.csv')
temp.to_csv(out_path, index_label='description')
temp

In [ ]:
temp = pd.read_csv(os.path.join(os.getcwd(), 'output_r', 'latex', 'mtl_vs_baseline.csv'))
temp.head()

## 6. Bootstrap

This section corresponds to the MTL bootstrap evaluation in Section 4.2.

* split datasets into Tbv and Ubv
* train model
* predict on Ubv
* join results with previous runs

returns
* index, predicted value

In [ ]:
def bootstrap_split(df, frac=1.0, random_state=None, replace=True):
    if random_state is None:
        df_train = df.sample(frac=frac, replace=replace)
    else:
        df_train = df.sample(frac=frac, replace=replace, random_state=random_state)
    df_test = df[~df.index.isin(df_train.index.unique())]
    return df_train, df_test


def get_bias(df, result):
    biases = []
    for k, v in df.iterrows():
        y_true = v[TARGET]
        y_preds = []
        for sim in result:
            try:
                y_preds.append(sim.loc[k])
            except:
                continue
        if len(y_preds) > 0:
            bias = np.abs(np.mean(y_preds) - y_true)
            biases.append(bias)
    return np.mean(biases), biases


def prepare_bootstrap_data(data, frac=1.0, replace=True):
    df_train_diesel, df_test_diesel = bootstrap_split(data['diesel'], frac=frac, replace=replace)
    df_train_hybrid, df_test_hybrid = bootstrap_split(data['hybrid'], frac=frac, replace=replace)
    df_train_electric, df_test_electric = bootstrap_split(data['electric'], frac=frac, replace=replace)
    
    datasets2 = {}
    datasets2['diesel_train'] = MyDataset(df_train_diesel[FEATURE_COLUMNS], df_train_diesel[TARGET])
    datasets2['diesel_test'] = MyDataset(df_test_diesel[FEATURE_COLUMNS], df_test_diesel[TARGET])
    datasets2['hybrid_train'] = MyDataset(df_train_hybrid[FEATURE_COLUMNS], df_train_hybrid[TARGET])
    datasets2['hybrid_test'] = MyDataset(df_test_hybrid[FEATURE_COLUMNS], df_test_hybrid[TARGET])
    datasets2['electric_train'] = MyDataset(df_train_electric[FEATURE_COLUMNS], df_train_electric[TARGET])
    datasets2['electric_test'] = MyDataset(df_test_electric[FEATURE_COLUMNS], df_test_electric[TARGET])
    
    datas = {'df_train_diesel': df_train_diesel, 
             'df_test_diesel': df_test_diesel, 
             'df_train_hybrid': df_train_hybrid, 
             'df_test_hybrid': df_test_hybrid, 
             'df_train_electric': df_train_electric, 
             'df_test_electric': df_test_electric}
    return datasets2, datas


def run_bootstrap_baseline_interation(datasets,
                                      datas,
                                      params, 
                                      device, 
                                      num_epochs=300, 
                                      early_stopping=False, 
                                      early_stop_epochs=1000, 
                                      stats_at_epoch=1, 
                                      frac=1.0):
    df_results, models = train_baseline_allclasses(datasets,
                                                   params['n_hidden'], 
                                                   params['n_layers'], 
                                                   params['lr'], 
                                                   params['batch_size'], 
                                                   num_epochs, 
                                                   device, 
                                                   dropout=0,
                                                   test=True,
                                                   vehicle_class_list=['diesel', 'hybrid', 'electric'],
                                                   stats_at_epoch=stats_at_epoch, 
                                                   early_stopping=early_stopping, 
                                                   early_stop_epochs=early_stop_epochs, 
                                                   metrics=[params['loss_fun']], 
                                                   loss_fun=params['loss_fun'], filename=params['filename'], val_on_train=False)
    models['diesel'].eval()
    models['hybrid'].eval()
    models['electric'].eval()
    with torch.no_grad():
        # predict values diesel
        features = torch.tensor(datas['df_test_diesel'][FEATURE_COLUMNS].values).double().to(device)
        predicts = models['diesel'](features).cpu().detach().numpy()
        result_diesel = pd.Series(data=predicts, index=datas['df_test_diesel'].index)
    
        # predict values hybrid
        features = torch.tensor(datas['df_test_hybrid'][FEATURE_COLUMNS].values).double().to(device)
        predicts = models['hybrid'](features).cpu().detach().numpy()
        result_hybrid = pd.Series(data=predicts, index=datas['df_test_hybrid'].index)
    
        # predict values electric
        features = torch.tensor(datas['df_test_electric'][FEATURE_COLUMNS].values).double().to(device)
        predicts = models['electric'](features).cpu().detach().numpy()
        result_electric = pd.Series(data=predicts, index=datas['df_test_electric'].index)
    
    return result_diesel, result_hybrid, result_electric
    
    


def run_bootstrap_multitask_interation(datasets, 
                                       datas,
                                       mtl_params, 
                                       device, 
                                       num_epochs=300, 
                                       early_stopping=False, 
                                       early_stop_epochs=1000, 
                                       stats_at_epoch=1):
    df_results, mtl_model = train_multitask(datasets, 
                                            mtl_params['n_shared_width'], 
                                            mtl_params['n_shared_depth'], 
                                            mtl_params['n_task_widths'], 
                                            mtl_params['lr'], 
                                            mtl_params['batch_size'], 
                                            num_epochs, device, 
                                            loss_fun=mtl_params['loss_fun'], 
                                            test=True, 
                                            stats_at_epoch=stats_at_epoch, 
                                            early_stopping=early_stopping, 
                                            outlayer=mtl_params['outlayer'], 
                                            early_stop_epochs=early_stop_epochs, 
                                            metrics=[mtl_params['loss_fun']], 
                                            opt=mtl_params['opt'], filename=mtl_params['filename'], val_on_train=False)
    mtl_model.eval()
    with torch.no_grad():
        # predict values diesel
        features = torch.tensor(datas['df_test_diesel'][FEATURE_COLUMNS].values).double().to(device)
        mtl_model.task = 'diesel'
        predicts = mtl_model(features).cpu().detach().numpy()
        result_diesel = pd.Series(data=predicts, index=datas['df_test_diesel'].index)
    
        # predict values hybrid
        features = torch.tensor(datas['df_test_hybrid'][FEATURE_COLUMNS].values).double().to(device)
        mtl_model.task = 'hybrid'
        predicts = mtl_model(features).cpu().detach().numpy()
        result_hybrid = pd.Series(data=predicts, index=datas['df_test_hybrid'].index)
    
        # predict values electric
        features = torch.tensor(datas['df_test_electric'][FEATURE_COLUMNS].values).double().to(device)
        mtl_model.task = 'electric'
        predicts = mtl_model(features).cpu().detach().numpy()
        result_electric = pd.Series(data=predicts, index=datas['df_test_electric'].index)
    return result_diesel, result_hybrid, result_electric


def run_bootstrap(data, 
                  num_boot_runs, 
                  device, 
                  mtl_params, 
                  baseline_params, 
                  num_epochs, 
                  frac=1.0, 
                  replace=True, 
                  model_type=['mtl', 'baseline']):
    results = {}
    if 'mtl' in model_type:
        results['mtl_diesel'] = []
        results['mtl_hybrid'] = []
        results['mtl_electric'] = []
    if 'baseline' in model_type:
        results['baseline_diesel'] = []
        results['baseline_hybrid'] = []
        results['baseline_electric'] = []
        
    for boot_run in range(num_boot_runs):
        start_time = time.time()
        datasets, datas = prepare_bootstrap_data(data, frac=frac, replace=replace)
        if 'mtl' in model_type:
            mtl_diesel, mtl_hybrid, mtl_electric = run_bootstrap_multitask_interation(deepcopy(datasets), deepcopy(datas), mtl_params, device, num_epochs=num_epochs)
            results['mtl_diesel'].append(mtl_diesel)
            results['mtl_hybrid'].append(mtl_hybrid)
            results['mtl_electric'].append(mtl_electric)
        if 'baseline' in model_type:
            baseline_diesel, baseline_hybrid, baseline_electric = run_bootstrap_baseline_interation(deepcopy(datasets), deepcopy(datas), baseline_params, device, num_epochs=num_epochs)
            results['baseline_diesel'].append(baseline_diesel)
            results['baseline_hybrid'].append(baseline_hybrid)
            results['baseline_electric'].append(baseline_electric)
        end_time = time.time() - start_time
        print(f"done with boot_run: {boot_run} in {end_time} seconds")
        out_path = os.path.join(os.getcwd(), "output_r", "latex", f"bootresultstemp2.pkl")
        with open(out_path, 'wb') as handle:
            pickle.dump(results, handle)
    return results


def get_ave_mse(data, result):
    mses = []
    for sim in result:
        y_true = data.loc[sim.index, TARGET].values
        y_pred = sim.values
        mses.append(sklearn.metrics.mean_squared_error(y_true, y_pred))
        #maes.append(sklearn.metrics.mean_absolute_error(y_true, y_pred))
    return np.mean(mses), mses


def get_ave_mae(data, result):
    maes = []
    for sim in result:
        y_true = data.loc[sim.index, TARGET].values
        y_pred = sim.values
        #mses.append(sklearn.metrics.mean_squared_error(y_true, y_pred))
        maes.append(sklearn.metrics.mean_absolute_error(y_true, y_pred))
    return np.mean(maes), maes
        

In [ ]:
mtl_params = {'loss_fun': 'mse', 'outlayer': 'linear', 'n_shared_width': 300, 'n_shared_depth': 5, 'n_task_widths': [64, 32, 16], 'lr': 0.0005, 'batch_size': 256, 'opt': 'adamw', 'filename': os.path.join(os.getcwd(), 'models', 'bootstrapmtl.pt')}
baseline_params = {'loss_fun': 'mse', 'n_hidden': 300, 'n_layers': 5, 'lr': 0.0005, 'batch_size': 256, 'filename': os.path.join(os.getcwd(), 'models', 'bootstrapbaseline.pt')}

num_boot_runs = 30
device = torch.device('cuda:2')
num_epochs = 150
model_type=['mtl', 'baseline']

data = {}
for vehicle_class in ['diesel', 'hybrid', 'electric']:
    features = pd.concat([datasets[f"{vehicle_class}_train"].features.copy(deep=True), datasets[f"{vehicle_class}_val"].features.copy(deep=True), datasets[f"{vehicle_class}_test"].features.copy(deep=True)], ignore_index=True)
    labels = pd.concat([datasets[f"{vehicle_class}_train"].labels.copy(deep=True), datasets[f"{vehicle_class}_val"].labels.copy(deep=True), datasets[f"{vehicle_class}_test"].labels.copy(deep=True)], ignore_index=True)
    features[TARGET] = labels
    data[vehicle_class] = features

In [ ]:
results = run_bootstrap(deepcopy(data), 
                        num_boot_runs, 
                        device, 
                        mtl_params, 
                        baseline_params, 
                        num_epochs, 
                        frac=1.0, 
                        replace=True, 
                        model_type=model_type)

out_path = os.path.join(os.getcwd(), "output_r", "latex", f"bootresults.pkl")
with open(out_path, 'wb') as handle:
    pickle.dump(results, handle)

saved_results = {}
for m in model_type:
    bias_diesel, biases_diesel = get_bias(data['diesel'], results[f"{m}_diesel"])
    bias_hybrid, biases_hybrid = get_bias(data['hybrid'], results[f"{m}_hybrid"])
    bias_electric, biases_electric = get_bias(data['electric'], results[f"{m}_electric"])
    
    ave_mae_diesel, maes_diesel = get_ave_mae(data['diesel'], results[f"{m}_diesel"])
    ave_mae_hybrid, maes_hybrid = get_ave_mae(data['hybrid'], results[f"{m}_hybrid"])
    ave_mae_electric, maes_electric = get_ave_mae(data['electric'], results[f"{m}_electric"])
    
    ave_mse_diesel, mses_diesel = get_ave_mse(data['diesel'], results[f"{m}_diesel"])
    ave_mse_hybrid, mses_hybrid = get_ave_mse(data['hybrid'], results[f"{m}_hybrid"])
    ave_mse_electric, mses_electric = get_ave_mse(data['electric'], results[f"{m}_electric"])
    
    saved_results[f"{m}_bias"] = [bias_diesel, bias_hybrid, bias_electric]
    saved_results[f"{m}_mae"] = [ave_mae_diesel, ave_mae_hybrid, ave_mae_electric]
    saved_results[f"{m}_mse"] = [ave_mse_diesel, ave_mse_hybrid, ave_mse_electric]
    
    box = {}
    quantiles = [0.01, 0.25, 0.5, 0.75, 0.99]
    box['diesel'] = pd.Series(biases_diesel).quantile(quantiles).tolist()
    box['hybrid'] = pd.Series(biases_hybrid).quantile(quantiles).tolist()
    box['electric'] = pd.Series(biases_electric).quantile(quantiles).tolist()
    box_results = pd.DataFrame(box)
    out_path = os.path.join(os.getcwd(), "output_r", "latex", f"full_boot_{m}_bias_distribution.csv")
    box_results.to_csv(out_path, index=False)

    mae_result = pd.DataFrame({'diesel': maes_diesel, 'hybrid': maes_hybrid, 'electric': maes_electric})
    out_path = os.path.join(os.getcwd(), "output_r", "latex", f"full_boot_{m}_mae_distribution.csv")
    mae_result.to_csv(out_path, index=False)

    mse_result = pd.DataFrame({'diesel': mses_diesel, 'hybrid': mses_hybrid, 'electric': mses_electric})
    out_path = os.path.join(os.getcwd(), "output_r", "latex", f"full_boot_{m}_mse_distribution.csv")
    mse_result.to_csv(out_path, index=False)
    
df_temp = pd.DataFrame(saved_results, index=['diesel', 'hybrid', 'electric'])
out_path = os.path.join(os.getcwd(), "output_r", "latex", "full_boot.csv")
df_temp.to_csv(out_path, index_label='vehicle')

## 8. ITL

This is the ITL evaluation presented in Section 4.3 of the paper.

In [ ]:
def itl_iteration(source, 
                  target, 
                  baseline_params, 
                  target_frac,
                  device,
                  feature_extract=True, 
                  source_frac=0.9, 
                  num_epochs=150, 
                  num_runs=10):
    
    results = {'source': [], 
               'target': [], 
               'target_frac': [], 
               'target_samples': [], 
               'feature_extract': [], 
               'mse_target': [], 
               'mae_target': [], 
               'mse_baseline': [], 
               'mae_baseline': []}
    
    for num_run in range(num_runs):
        source_datasets, source_datas = prepare_bootstrap_data(data, frac=source_frac, replace=False)
        target_datasets, target_datas = prepare_bootstrap_data(data, frac=target_frac, replace=False)

        # first train a baseline - source
        df_results_source, model = train_baseline(source_datasets,
                                                  source,
                                                  baseline_params['n_hidden'], 
                                                  baseline_params['n_layers'], 
                                                  baseline_params['lr'], 
                                                  baseline_params['batch_size'], 
                                                  num_epochs, 
                                                  device, 
                                                  dropout=0,
                                                  test=True, 
                                                  stats_at_epoch=1, 
                                                  early_stopping=False, 
                                                  early_stop_epochs=1000, 
                                                  metrics=['mse', 'mae', 'huber'], 
                                                  loss_fun=baseline_params['loss_fun'], 
                                                  filename=os.path.join(os.getcwd(), 'models', 'itlsource.pt'), 
                                                  val_on_train=False)
    
    
        if feature_extract:
            params_to_update = []
            for name, param in model.named_parameters():
                temp = name.split('.')[0]
                if temp in ['v1', 'v2', 'v3', 'out']:
                    params_to_update.append(param)
                else:
                    param.requires_grad = False
                #if param.requires_grad == True:
                #    params_to_update.append(param)
        else:
            params_to_update = model.parameters()
    
        if baseline_params['opt'] == 'adam':
            optimizer = torch.optim.Adam(params_to_update, lr=baseline_params['lr'])
        else:
            optimizer = torch.optim.AdamW(params_to_update, lr=baseline_params['lr'])
        
        # train target
        df_results_target, model = train_baseline(target_datasets,
                                                  target,
                                                  baseline_params['n_hidden'], 
                                                  baseline_params['n_layers'], 
                                                  baseline_params['lr'], 
                                                  baseline_params['batch_size'], 
                                                  num_epochs, 
                                                  device, 
                                                  dropout=0,
                                                  test=True, 
                                                  stats_at_epoch=1, 
                                                  early_stopping=False, 
                                                  early_stop_epochs=1000, 
                                                  metrics=['mse', 'mae', 'huber'], 
                                                  loss_fun=baseline_params['loss_fun'], 
                                                  filename=os.path.join(os.getcwd(), 'models', 'itltarget.pt'), 
                                                  val_on_train=False,
                                                 model=model, 
                                                  optimizer=optimizer)




        # train baseline

        baseline_model = BaselineModel(len(FEATURE_COLUMNS), baseline_params['n_hidden'], 1, 0, baseline_params['n_layers']).to(device)
        if baseline_params['opt'] == 'adam':
            baseline_optimizer = torch.optim.Adam(baseline_model.parameters(), lr=baseline_params['lr'])
        else:
            baseline_optimizer = torch.optim.AdamW(baseline_model.parameters(), lr=baseline_params['lr'])

    
        df_results_baseline, baseline_model = train_baseline(target_datasets,
                                                            target,
                                                              baseline_params['n_hidden'], 
                                                              baseline_params['n_layers'], 
                                                              baseline_params['lr'], 
                                                              baseline_params['batch_size'], 
                                                              num_epochs, 
                                                              device, 
                                                              dropout=0,
                                                              test=True, 
                                                              stats_at_epoch=1, 
                                                          early_stopping=False, 
                                                          early_stop_epochs=1000, 
                                                          metrics=['mse', 'mae', 'huber'], 
                                                          loss_fun=baseline_params['loss_fun'], 
                                                          filename=os.path.join(os.getcwd(), 'models', 'itlbase.pt'), 
                                                          val_on_train=False,
                                                         model=baseline_model, 
                                                         optimizer=baseline_optimizer)
    
        results['source'].append(source)
        results['target'].append(target)
        results['target_frac'].append(target_frac)
        results['target_samples'].append(len(target_datas[f"df_train_{target}"]))
        results['feature_extract'].append(feature_extract)
        results['mse_target'].append(df_results_target['test_mse'].min())
        results['mae_target'].append(df_results_target['test_mae'].min())
        results['mse_baseline'].append(df_results_baseline['test_mse'].min())
        results['mae_baseline'].append(df_results_baseline['test_mae'].min())
    return pd.DataFrame(results)

In [ ]:
baseline_params = {'opt': 'adam', 'loss_fun': 'mse', 'n_hidden': 300, 'n_layers': 5, 'lr': 0.0005, 'batch_size': 256}
target_fracs = [0.02, 0.05, 0.1, 0.15]
device = torch.device('cuda:1')
results = []
vehicles = ['diesel', 'hybrid', 'electric']
for source in vehicles:
    for target in vehicles:
        if source != target:
            for target_frac in target_fracs:
                #out_path = os.path.join(os.getcwd(), "output_r", "latex", "itl_temp.csv")
                #temp = pd.read_csv(out_path)
                #if len(temp[(temp['source']==source) & (temp['target']==target) & (temp['target_frac']==target_frac)]) > 0:
                #    print('continue')
                #    continue
                start_time = time.time()
                df = itl_iteration(source, target, baseline_params, target_frac, device)
                results.append(df)
                end_time = time.time() - start_time
                print(f"Done with source: {source}, target: {target}, target_frac: {target_frac}, took {end_time} seconds")
                if len(results) > 1:
                    out_path = os.path.join(os.getcwd(), "output_r", "latex", "itl_temp.csv")
                    temp = pd.concat(results, ignore_index=True)
                    temp.to_csv(out_path, index=False)
                
out_path = os.path.join(os.getcwd(), "output_r", "latex", "itl.csv")
temp = pd.concat(results, ignore_index=True)
temp.to_csv(out_path, index=False)

In [ ]:
datasets, datas = prepare_bootstrap_data(data, frac=0.8, replace=False)

## 9. t-SNE Visualization

This corresponds with the discussion Section 4.4.

In [ ]:
# diesel
num_epochs = 20
device = torch.device('cuda:2')
baseline_params = {'opt': 'adamw', 'loss_fun': 'mse', 'n_hidden': 300, 'n_layers': 5, 'lr': 0.0005, 'batch_size': 256}

datasets, datas = prepare_bootstrap_data(data, frac=0.8, replace=False)
#target_datasets, target_datas = prepare_bootstrap_data(data, frac=target_frac, replace=False)

# diesel
df_results_diesel, model_diesel = train_baseline(datasets,
                                            'diesel',
                                                  baseline_params['n_hidden'], 
                                                  baseline_params['n_layers'], 
                                                  baseline_params['lr'], 
                                                  baseline_params['batch_size'], 
                                                  num_epochs, 
                                                  device, 
                                                  dropout=0,
                                                  test=True, 
                                                  stats_at_epoch=1, 
                                                  early_stopping=False, 
                                                  early_stop_epochs=1000, 
                                                  metrics=['mse', 'mae', 'huber'], 
                                                  loss_fun=baseline_params['loss_fun'], 
                                                  filename=os.path.join(os.getcwd(), 'models', 'tnsediesel.pt'), 
                                                  val_on_train=False)

df_results_hybrid, model_hybrid = train_baseline(datasets,
                                            'hybrid',
                                                  baseline_params['n_hidden'], 
                                                  baseline_params['n_layers'], 
                                                  baseline_params['lr'], 
                                                  baseline_params['batch_size'], 
                                                  num_epochs, 
                                                  device, 
                                                  dropout=0,
                                                  test=True, 
                                                  stats_at_epoch=1, 
                                                  early_stopping=False, 
                                                  early_stop_epochs=1000, 
                                                  metrics=['mse', 'mae', 'huber'], 
                                                  loss_fun=baseline_params['loss_fun'], 
                                                  filename=os.path.join(os.getcwd(), 'models', 'tnsehybrid.pt'), 
                                                  val_on_train=False)

df_results_electric, model_electric = train_baseline(datasets,
                                            'electric',
                                                  baseline_params['n_hidden'], 
                                                  baseline_params['n_layers'], 
                                                  baseline_params['lr'], 
                                                  baseline_params['batch_size'], 
                                                  num_epochs, 
                                                  device, 
                                                  dropout=0,
                                                  test=True, 
                                                  stats_at_epoch=1, 
                                                  early_stopping=False, 
                                                  early_stop_epochs=1000, 
                                                  metrics=['mse', 'mae', 'huber'], 
                                                  loss_fun=baseline_params['loss_fun'], 
                                                  filename=os.path.join(os.getcwd(), 'models', 'tnseelectric.pt'), 
                                                  val_on_train=False)

In [ ]:
# load the models

# diesel
model_path = os.path.join(os.getcwd(), 'models', 'tnsediesel.pt')
device = torch.device('cuda:2')
model_diesel = BaselineModel(len(FEATURE_COLUMNS), 300, 1, 0, 5)
#baseline_model = MultiTaskModel(len(FEATURE_COLUMNS), 300, 5, [64, 32, 16], outlayer='linear')
checkpoint = torch.load(model_path)
model_diesel.load_state_dict(checkpoint['model_state_dict'])
model_diesel.double()
model_diesel.to(device)

# hybrid
model_path = os.path.join(os.getcwd(), 'models', 'tnsehybrid.pt')
device = torch.device('cuda:2')
model_hybrid = BaselineModel(len(FEATURE_COLUMNS), 300, 1, 0, 5)
#baseline_model = MultiTaskModel(len(FEATURE_COLUMNS), 300, 5, [64, 32, 16], outlayer='linear')
checkpoint = torch.load(model_path)
model_hybrid.load_state_dict(checkpoint['model_state_dict'])
model_hybrid.double()
model_hybrid.to(device)

# electric
model_path = os.path.join(os.getcwd(), 'models', 'tnseelectric.pt')
device = torch.device('cuda:2')
model_electric = BaselineModel(len(FEATURE_COLUMNS), 300, 1, 0, 5)
#baseline_model = MultiTaskModel(len(FEATURE_COLUMNS), 300, 5, [64, 32, 16], outlayer='linear')
checkpoint = torch.load(model_path)
model_electric.load_state_dict(checkpoint['model_state_dict'])
model_electric.double()
model_electric.to(device)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:

#quantiles = [0.01, 0.2, 0.4, 0.6, 0.8, 0.99]
#colors = ['midnightblue', 'mediumblue', 'mediumpurple', 'violet', 'crimson']

def get_color(x, quant, colors):
    result = None
    for i in range(1, len(quant)):
        if (x >= quant[i-1]) & (x < quant[i]):
            result = colors[i-1]
    return result

(fig, subplots) = plt.subplots(3, 3, figsize=(15, 10.5))

vehicle_types = ['diesel', 'hybrid', 'electric']
for i in range(len(vehicle_types)):
    source = vehicle_types[i]
    if source == 'diesel':
        model = model_diesel
    elif source == 'hybrid':
        model = model_hybrid
    else:
        model = model_electric
    for j in range(len(vehicle_types)):
        target = vehicle_types[j]
        df = datas[f"df_train_{target}"]
        quant = df['target_kg'].quantile([0.1, 0.9]).tolist()  
        norm = Normalize(vmin=quant[0], vmax=quant[-1], clip=True)
        mapper = cm.ScalarMappable(norm=norm, cmap=cm.plasma)
        samples = df.sample(frac=.02, replace=False, random_state=100)
        samples['color'] = samples['target_kg'].apply(lambda x: mapper.to_rgba(x))
        #samples = samples.dropna()
        features = torch.tensor(samples[FEATURE_COLUMNS].values).double().to(device)
        predicts = model.feature_extractor(features).cpu().detach().numpy()
        X = TSNE(n_components=2, perplexity=10, init='pca').fit_transform(predicts)
        
        ax = subplots[i][j]
        if source == 'diesel':
            title1 = 'ICEV'
        elif source == 'hybrid':
            title1 = 'HV'
        else:
            title1 = 'EV'
        if target == 'diesel':
            title2 = 'ICEV'
        elif target == 'hybrid':
            title2 = 'HV'
        else:
            title2 = 'EV'
        ax.set_title(f"Source: {title1}, Target: {title2}", fontsize=18)
        ax.scatter(X[:,0], X[:,1], c=samples['color'])
        ax.xaxis.set_major_formatter(NullFormatter())
        ax.yaxis.set_major_formatter(NullFormatter())
        ax.axis('tight')
plt.subplots_adjust(hspace = .3)
plt.show()

In [ ]:

#quantiles = [0.01, 0.2, 0.4, 0.6, 0.8, 0.99]
#colors = ['midnightblue', 'mediumblue', 'mediumpurple', 'violet', 'crimson']

def get_color(x, quant, colors):
    result = None
    for i in range(1, len(quant)):
        if (x >= quant[i-1]) & (x < quant[i]):
            result = colors[i-1]
    return result

(fig, subplots) = plt.subplots(1, 3, figsize=(15, 3))

vehicle_types = ['diesel', 'hybrid', 'electric']
for i in range(len(vehicle_types)):
    source = vehicle_types[i]
    if source == 'diesel':
        model = model_diesel
    elif source == 'hybrid':
        model = model_hybrid
    else:
        model = model_electric
    #for j in range(len(vehicle_types)):
    target = source
    df = datas[f"df_train_{target}"]
    quant = df['target_kg'].quantile([0.1, 0.9]).tolist()  
    norm = Normalize(vmin=quant[0], vmax=quant[-1], clip=True)
    mapper = cm.ScalarMappable(norm=norm, cmap=cm.plasma)
    samples = df.sample(frac=.02, replace=False, random_state=100)
    samples['color'] = samples['target_kg'].apply(lambda x: mapper.to_rgba(x))

    X = TSNE(n_components=2, perplexity=10, init='pca').fit_transform(samples[FEATURE_COLUMNS].values)
        
    ax = subplots[i]
    if source == 'diesel':
        title1 = 'ICEV'
    elif source == 'hybrid':
        title1 = 'HV'
    else:
        title1 = 'EV'
    ax.set_title(title1, fontsize=18)
    ax.scatter(X[:,0], X[:,1], c=samples['color'])
    ax.xaxis.set_major_formatter(NullFormatter())
    ax.yaxis.set_major_formatter(NullFormatter())
    ax.axis('tight')
#plt.subplots_adjust(hspace = .3)
plt.show()

In [ ]:
out_path = os.path.join(os.getcwd(), "output_r", "images", "tsnefeatureextract.png")
fig.savefig(out_path)

In [ ]:
plt.scatter(X[:,0], X[:,1], c=samples['quantile'])

In [ ]:
df = datas["df_train_diesel"]
minima = df['target_kg'].min()
maxima = df['target_kg'].max()
print(minima, maxima)

norm = Normalize(vmin=minima, vmax=maxima, clip=True)
mapper = cm.ScalarMappable(norm=norm, cmap=cm.plasma)

In [ ]:
mapper.to_rgba(0.005)